In [17]:
from pprint import pprint
import requests
from bs4 import BeautifulSoup
from htmldate import find_date
import numpy as np
import random as rd
from lxml.html import fromstring
from itertools import cycle
import time
from datetime import datetime
import os
from googleapiclient.discovery import build 
import json
import traceback

In [ ]:
from googlesearch import search

In [ ]:
NUM_OF_RESULTS = 100
DATA_DIR = 'data'
OUT_DIR = 'out'
AUTH_DIR = 'C:\\PycharmProjects\\universities_covid19_auth.txt'
sep = os.sep

In [ ]:
universities = [u.strip() for u in open(f'{DATA_DIR}{sep}Universities.txt').readlines()]

In [ ]:
api_key, cse_id = [l.split('=')[1].strip() for l in open(AUTH_DIR).readlines()]

In [ ]:
def google_query(query, api_key, cse_id, **kwargs):
    query_service = build("customsearch", 
                          "v1", 
                          developerKey=api_key
                          )  
    query_results = query_service.cse().list(q=query,    # Query
                                             cx=cse_id,  # CSE ID
                                             **kwargs    
                                             ).execute()
    return query_results['items']

In [ ]:
universitites_search = {}
for uni in universities:
    qry = f'{uni} (COVID-19| COVID 19| COVID19 | Coronavirus 2019)'
    my_results = google_query(qry, api_key, cse_id, num = 10)
    
    universitites_search[uni] = []
    for result in my_results:
        universitites_search[uni].append(result['link'])

In [27]:
text_data = []
for uni, urls in universitites_search.items():
    with open(f'{OUT_DIR}{os.sep}{uni}.csv', 'w', encoding="utf-8") as wr:
        for url in urls:
           
            try:
                url = url.split('#')[0].replace('www.', '')

                page = requests.get(url, timeout=5)
                if page.status_code != 200:
                    continue

                soup = BeautifulSoup(page.content, 'html.parser')
                
                for script in soup(["script", "style"]):
                    script.extract()

                text = soup.get_text()
                lines = (line.strip() for line in text.splitlines())
                chunks = (phrase.strip() for line in lines for phrase in line.split("  "))
                text = '\n'.join(chunk for chunk in chunks if chunk)
                text_data.append(text)
            except Exception as e:
                traceback.print_exc()

KeyboardInterrupt: 

In [30]:
from sklearn.feature_extraction.text import TfidfVectorizer

In [31]:
vec = TfidfVectorizer()

In [51]:
from nltk.corpus import stopwords
nltk.download('stopwords')
from nltk.tokenize import word_tokenize
nltk.download('punkt')

[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\sraas\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\sraas\AppData\Roaming\nltk_data...
[nltk_data]   Unzipping tokenizers\punkt.zip.


True

In [62]:
text_tokens = []
for text in text_data:
    text_tokens.append(' '.join(word_tokenize(text)))

In [63]:
x = vec.fit_transform(text_tokens)

In [66]:
vec.get_feature_names()

['00',
 '000',
 '000655',
 '000680',
 '0038',
 '00am',
 '00pm',
 '0123',
 '0123member',
 '01pm',
 '02',
 '020',
 '0206',
 '02139',
 '02142',
 '02421',
 '0246',
 '0290',
 '02pm',
 '03',
 '0312',
 '04',
 '04pm',
 '05',
 '06',
 '061',
 '06510',
 '06510maps',
 '06511',
 '06520',
 '0656',
 '07',
 '08',
 '10',
 '100',
 '1083ago',
 '11',
 '1111',
 '11935',
 '12',
 '125',
 '13',
 '1311',
 '1332',
 '13th',
 '14',
 '15',
 '150',
 '16',
 '160',
 '17',
 '1700',
 '17005',
 '175',
 '17621',
 '18',
 '180',
 '18th',
 '19',
 '1906',
 '1918',
 '195',
 '19a',
 '19academic',
 '19georgia',
 '19guidance',
 '19your',
 '1b',
 '1v8vypzwchgtue_wybgcrzcn0qqu1vwfy1ngvqxuxs',
 '20',
 '200',
 '2000',
 '2002',
 '2010',
 '2012',
 '2013',
 '2014',
 '2015',
 '2016',
 '2017',
 '2018',
 '2019',
 '2019covid',
 '2019n',
 '2019ncov',
 '2020',
 '2020by',
 '2020calendarcontact',
 '2020facebook',
 '2020guidance',
 '2020statement',
 '2020submitted',
 '2020tagsinfectious',
 '2021',
 '2025',
 '203',
 '2050',
 '208237',
 '20th',
 